## Modules to import:

In [4]:
from ase.atoms import Atoms, Atom
from ase.io import read,write
from ase.constraints import FixInternals
from ase.visualize import view
from ase.optimize import BFGS
from ase.neb import NEB
from math import pi
import obcalc
import os,shutil

In [7]:
try:
    os.mkdir('AIMS_ASE_NEB'); os.chdir('AIMS_ASE_NEB')
except:
    print('Folder already exists'); os.chdir('AIMS_ASE_NEB'); print(os.getcwd())

Folder already exists


In [10]:
for i in range(1,7):
    try:
        os.mkdir(str(i))
    except:
        print(str(i)+' - directory already exists!')
    shutil.copy('../'+str(i)+'/cis/optimized.xyz', str(i)+'/000.xyz')
    shutil.copy('../'+str(i)+'/trans/optimized.xyz', str(i)+'/180.xyz')

In [27]:
os.chdir('../6')
hti = read('000.xyz')
d_indices = [12,7,14,15]
angle = ["%03d" % i for i in range(0,190,15)]
if len(hti.get_atomic_numbers()) == 27:
    mask = np.zeros(14).tolist() + np.ones(13).tolist()
if len(hti.get_atomic_numbers()) > 27:
    mask = np.zeros(14).tolist() + np.ones(13).tolist() + np.zeros(len(hti.get_atomic_numbers())-27).tolist()
if len(hti.get_atomic_numbers()) == 34:
    mask = np.zeros(14).tolist() + np.ones(13).tolist() + np.zeros(len(hti.get_atomic_numbers())-31).tolist() + np.ones(4).tolist()
for i in angle:
    hti.set_dihedral(d_indices, int(i)*pi/180, mask)
    dmasses=[hti.get_masses()[d_indices[0]],hti.get_masses()[d_indices[1]],hti.get_masses()[d_indices[2]],hti.get_masses()[d_indices[3]]]
    dihedral_constraint = [hti.get_dihedral(d_indices), d_indices, dmasses]
    constraint = FixInternals(dihedrals=[dihedral_constraint])
    hti.set_constraint(constraint)
    hti.set_calculator(obcalc.OBForceField('uff', bonds=obcalc.get_bonds(hti)))
    relax = BFGS(hti)
    relax.run(fmax=0.1)
    hti.write('path-'+str(i)+'.xyz')

      Step     Time          Energy         fmax
BFGS:    0 11:16:37        2.859673        3.9745
BFGS:    1 11:16:37        2.594707        1.5484
BFGS:    2 11:16:37        2.512184        0.8881
BFGS:    3 11:16:37        2.471215        0.5755
BFGS:    4 11:16:37        2.442617        0.4252
BFGS:    5 11:16:37        2.428838        0.4237
BFGS:    6 11:16:37        2.420929        0.2380
BFGS:    7 11:16:37        2.413005        0.2607
BFGS:    8 11:16:37        2.406945        0.2770
BFGS:    9 11:16:37        2.402159        0.1666
BFGS:   10 11:16:37        2.396572        0.1934
BFGS:   11 11:16:37        2.392122        0.2448
BFGS:   12 11:16:37        2.388962        0.1311
BFGS:   13 11:16:37        2.386642        0.1090
BFGS:   14 11:16:37        2.384845        0.1059
BFGS:   15 11:16:37        2.383429        0.0996
      Step     Time          Energy         fmax
BFGS:    0 11:16:37        2.784494        1.6744
BFGS:    1 11:16:37        2.704357        1.3322
BF

BFGS:    1 11:16:40        2.465484        1.4210
BFGS:    2 11:16:40        2.381919        1.1459
BFGS:    3 11:16:40        2.340815        0.9018
BFGS:    4 11:16:40        2.323820        0.3815
BFGS:    5 11:16:40        2.306910        0.4433
BFGS:    6 11:16:40        2.286997        0.6462
BFGS:    7 11:16:40        2.266482        0.2948
BFGS:    8 11:16:40        2.253461        0.3454
BFGS:    9 11:16:40        2.243882        0.3143
BFGS:   10 11:16:40        2.234613        0.2266
BFGS:   11 11:16:40        2.225037        0.1939
BFGS:   12 11:16:40        2.215819        0.2363
BFGS:   13 11:16:40        2.207504        0.2910
BFGS:   14 11:16:40        2.199367        0.2305
BFGS:   15 11:16:40        2.191037        0.1798
BFGS:   16 11:16:40        2.183126        0.2276
BFGS:   17 11:16:40        2.176845        0.1847
BFGS:   18 11:16:40        2.171883        0.1611
BFGS:   19 11:16:40        2.167621        0.1333
BFGS:   20 11:16:40        2.164106        0.1445


In [28]:
angle = ["%03d" % i for i in range(0,190,15)]
path = [read('000.xyz')]
for i in angle[1:-1]:
    path += [read('path-'+str(i)+'.xyz')]
path += [read('180.xyz')]

view(path)

In [38]:
%%writefile gasphase-neb.py
from ase.atoms import Atoms, Atom
from ase.io import read,write
from ase.constraints import FixInternals
from ase.visualize import view
from ase.optimize import BFGS
from ase.neb import NEB
from math import pi
from ase.calculators.aims import Aims
import os,shutil

angle = ["%03d" % i for i in range(0,190,15)]
path = [read('000.xyz')]
for i in angle[1:-1]:
    path += [read('path-'+str(i)+'.xyz')]
path += [read('180.xyz')]

basis = 'light'
hpc = 'TINIS'

for i,image in enumerate(path):
        calc = Aims(outfilename = "aims{0:1d}.out".format(i),
                aims_command = 'srun '+os.getenv("HOME")+'/software/FHIaims2020-200112_1/bin/tinis.ipi.aims.200112.scalapack.mpi.x',
                species_dir = os.getenv("HOME")+'/software/fhiaims/code/species_defaults/'+basis,
                xc = 'pbe',
                sc_iter_limit = 100,
                spin = 'none'
                relativistic = ['atomic_zora', 'scalar'],
                sc_accuracy_etot = 1e-6,
                sc_accuracy_eev = 0.001,
                sc_accuracy_rho = 1e-4,
                sc_accuracy_forces = 1e-3,
                mixer = 'pulay',
                KS_method = 'elpa',
               )
        image.set_calculator(calc)
        
neb = NEB(path, climb=False)
optimizer = BFGS(neb, trajectory = 'htineb.traj')
optmizer.run(fmax=0.2)


Writing gasphase-neb.py
